In [20]:
import copy
import pickle
from pprint import pprint
from pathlib import Path
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps as cm
from thalamocortex.utils import create_data_loaders, train_one_epoch_thalreadout
from thalamocortex.models import CTCNet, CTCNetThalReadout
from sklearn.decomposition import PCA
from scipy.special import softmax

load trained model

In [9]:
model_path = Path("/Users/patmccarthy/Documents/thalamocortex/results/21_02_24_mod1_leftrightmnist_gridsearch2/0_CTCNet_TC_multi_pre_activation_reciprocal/model.pth")

In [10]:
model = torch.load(model_path)

load model and training report

In [11]:
# training progress and hyperparameter paths
learning_path = Path("/Users/patmccarthy/Documents/thalamocortex/results/21_02_24_mod1_leftrightmnist_gridsearch2/0_CTCNet_TC_multi_pre_activation_reciprocal/learning.pkl")
params_path = Path("/Users/patmccarthy/Documents/thalamocortex/results/21_02_24_mod1_leftrightmnist_gridsearch2/0_CTCNet_TC_multi_pre_activation_reciprocal/hyperparams.pkl")

In [12]:
# load training progress
with open(learning_path, "rb") as handle:
    learning = pickle.load(handle)
    print(f"{learning.keys()=}")
results = {"val_losses": learning["val_losses"],
                        "train_losses": learning["train_losses"],
                        "train_time": learning["train_time"],
                        "state_dicts": learning["state_dicts"]}

learning.keys()=dict_keys(['train_losses', 'val_losses', 'final_val_losses', 'state_dicts', 'train_time'])


In [13]:
# load hyperparams
with open(params_path, "rb") as handle:
    hp = pickle.load(handle)

In [14]:
hp

{'norm': 'normalise',
 'dataset': 'LeftRightMNIST',
 'save_path': '/Users/patmccarthy/Documents/thalamocortex/data',
 'batch_size': 32,
 'input_size': 1568,
 'output_size': 10,
 'ctx_layer_size': 64,
 'thal_layer_size': 16,
 'thalamocortical_type': 'multi_pre_activation',
 'thal_reciprocal': True,
 'thal_to_readout': False,
 'thal_per_layer': False,
 'lr': 5e-06,
 'loss': CrossEntropyLoss(),
 'epochs': 800,
 'ohe_targets': True,
 'track_loss_step': 50}

In [15]:
# instantiate model
model = CTCNet(input_size=hp["input_size"],
                output_size=hp["output_size"],
                ctx_layer_size=hp["ctx_layer_size"],
                thal_layer_size=hp["thal_layer_size"],
                thalamocortical_type=hp["thalamocortical_type"],
                thal_reciprocal=hp["thal_reciprocal"],
                thal_to_readout=hp["thal_to_readout"], 
                thal_per_layer=hp["thal_per_layer"])

In [16]:
# get model trained to specified epoch
epoch = 200
weights = results["state_dicts"][0]

In [17]:
weights

OrderedDict([('thal.0.weight',
              tensor([[-0.0165, -0.0100,  0.0353,  ...,  0.0268,  0.0541, -0.0005],
                      [ 0.0515, -0.0023,  0.0343,  ..., -0.0538,  0.0523,  0.0534],
                      [ 0.0336,  0.0677,  0.0768,  ...,  0.0459, -0.0273, -0.0043],
                      ...,
                      [-0.0495,  0.0871, -0.0480,  ...,  0.0597,  0.0214,  0.0852],
                      [ 0.0778,  0.0143,  0.1628,  ...,  0.0449, -0.0094,  0.0383],
                      [-0.0070,  0.0341,  0.0703,  ...,  0.0381, -0.0492, -0.0921]])),
             ('thal.0.bias',
              tensor([ 0.0593,  0.0063, -0.0402,  0.0679,  0.2337, -0.0203,  0.1267,  0.0372,
                      -0.0227, -0.0600,  0.0614,  0.0982,  0.0622, -0.0260,  0.1111,  0.0560])),
             ('thal_to_ctx1_projections.weight',
              tensor([[-0.2014, -0.2006, -0.2079,  ...,  0.0336, -0.1844,  0.1968],
                      [ 0.1322, -0.2155,  0.0631,  ..., -0.2208,  0.1441,  0.0158]

In [18]:
# set model weights
model.load_state_dict(weights)

<All keys matched successfully>

create new model with thalamic readout

In [ ]:
hp_model_thal = copy.deepcopy(hp)
hp_model_thal["thal_output_size"] = 2

In [21]:
hp

{'norm': 'normalise',
 'dataset': 'LeftRightMNIST',
 'save_path': '/Users/patmccarthy/Documents/thalamocortex/data',
 'batch_size': 32,
 'input_size': 1568,
 'output_size': 10,
 'ctx_layer_size': 64,
 'thal_layer_size': 16,
 'thalamocortical_type': 'multi_pre_activation',
 'thal_reciprocal': True,
 'thal_to_readout': False,
 'thal_per_layer': False,
 'lr': 5e-06,
 'loss': CrossEntropyLoss(),
 'epochs': 800,
 'ohe_targets': True,
 'track_loss_step': 50}

In [ ]:
model_thal = CTCNetThalReadout(input_size=hp["input_size"],
                               output_size=hp["output_size"],
                               ctx_layer_size=hp["ctx_layer_size"],
                               thal_layer_size=hp["thal_layer_size"],
                               thalamocortical_type=hp["thalamocortical_type"],
                               thal_reciprocal=hp["thal_reciprocal"],
                               thal_to_readout=hp["thal_to_readout"], 
                               thal_per_layer=hp["thal_per_layer"],
                               thal_output_size=)

get training dataset

In [ ]:
# generate new targets with specified class as one and all others as zero
one_class = 1

initialise backbone with pretrained weights